In [1]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)
import sys
import datetime
import backtrader as bt
import backtrader.analyzers as btanalyzers

class CommInfoFractional(bt.CommissionInfo):
    def getsize(self, price, cash):
        '''Returns fractional size for cash operation @price'''
        return self.p.leverage * (cash / price)

### Buy and HODL
class BaselineStrategy(bt.Strategy):
    
    
    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close

        # To keep track of pending orders
        self.order = None

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log('BUY EXECUTED, %.2f' % order.executed.price)
            elif order.issell():
                self.log('SELL EXECUTED, %.2f' % order.executed.price)

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None

    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # Check if we are in the market
        if not self.position:
            cash = self.broker.get_cash()
            limit_price = self.data.close[0] * 1.01
            size = (cash * 0.95) / limit_price
            print(cash, self.data.close[0], limit_price, size)
            self.buy(exectype=bt.Order.Limit, size=size)

cerebro = bt.Cerebro()

from_date=datetime.datetime(2016, 1, 1)
to_date = datetime.datetime(2020, 12, 30)

# Create data feeds
data_ticks = bt.feeds.CCXT(exchange='coinbasepro', symbol='BTC/USD', name="btc_usd_tick",
                           timeframe=bt.TimeFrame.Days, compression=1, fromdate=from_date, todate=to_date)
cerebro.adddata(data_ticks)

cerebro.broker.setcash(10000.0)
cerebro.broker.addcommissioninfo(CommInfoFractional())
cerebro.broker.setcommission(commission=0.005, margin=False, mult=1.0, commtype=None, percabs=True, stocklike=False, interest=0.0, interest_long=False, leverage=1.0, automargin=False, name=None)

cerebro.addanalyzer(btanalyzers.SharpeRatio, _name='mysharpe')
cerebro.addanalyzer(btanalyzers.AnnualReturn, _name='annualreturn')
cerebro.addanalyzer(btanalyzers.DrawDown, _name='drawdown')
cerebro.addanalyzer(btanalyzers.Returns, _name='returns')
cerebro.addanalyzer(btanalyzers.TradeAnalyzer, _name='tradeanalyzer')

cerebro.addobserver(bt.observers.DrawDown)
# cerebro.addobserver(bt.observers.LogReturns)
# cerebro.addobserver(bt.observers.TimeReturn)

# Add the strategy
cerebro.addstrategy(BaselineStrategy)

# Run the strategy
strats = cerebro.run()

AttributeError: module 'backtrader.feeds' has no attribute 'CCXT'

In [2]:
print('Returns:', strats[0].analyzers.returns.pprint())
print('Sharpe Ratio:', strats[0].analyzers.mysharpe.pprint())
print('Annual Return:', strats[0].analyzers.annualreturn.pprint())
print('Drawdown:', strats[0].analyzers.drawdown.pprint())
print('Trade Analyzer:', strats[0].analyzers.tradeanalyzer.pprint())
cerebro.plot()

NameError: name 'strats' is not defined